In [17]:
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
from citipy import citipy
import requests
import requests
import time
from datetime import datetime

#from config import weather_api_key
import os
from dotenv import load_dotenv
load_dotenv()
weather_api_key = os.getenv('WEATHER_API_KEY')
g_key = os.getenv('G_KEY')

In [18]:
# Create a set of random latitude and longitudes combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-90.000, high=90.000, size=2000)
lats_lngs = zip(lats, lngs)
lats_lngs

In [19]:
# Creates variable to hold list of coordinates
coordinates = list(lats_lngs)
coordinates

[(-89.59045781514656, -78.84579873985757),
 (-12.582426551979296, -29.092776740844805),
 (-61.168483274310375, -76.41613783189545),
 (-15.474600770999302, 10.40278101904022),
 (40.19771298728918, -73.62378393058175),
 (20.312941454121514, -83.30018138032565),
 (-0.023853453917112688, -70.74384770896734),
 (21.39978337699698, 53.7077128057571),
 (-36.37883714452249, 85.9804541900545),
 (82.51425500322219, -46.90933294845141),
 (23.47100452256923, -65.04039217243464),
 (-73.57286558476045, 50.228457821661095),
 (56.678638021584646, -26.081967731763683),
 (47.6395608058541, 86.60558044516353),
 (68.97975160724312, -22.194956436018515),
 (83.79257788930227, 56.6062804652681),
 (-3.0970541749015297, -0.07832361920719677),
 (-23.60716331428401, -1.1034013145195303),
 (36.606676663094845, -55.63725912277364),
 (-61.13105997360592, 0.8392899692698705),
 (44.19190203658803, -3.769334895058819),
 (-83.89314854026239, -32.7017996965736),
 (42.5341747211061, 64.32565030925164),
 (53.67604848505482

In [20]:
# Creates a list to hold cities.
cities = []

In [21]:
# Locates nearest city to our randomly generated list of coordinates using citipy.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # Only add unique cities to the list
    if city not in cities:
        cities.append(city)
        
# City count to confirm sufficient cities added.
cities

['ushuaia',
 'maragogi',
 'punta arenas',
 'namibe',
 'long branch',
 'santa fe',
 'mitu',
 'abu dhabi',
 'busselton',
 'upernavik',
 'road town',
 'port alfred',
 'grindavik',
 'altay',
 'illoqqortoormiut',
 'belushya guba',
 'takoradi',
 'jamestown',
 'saint george',
 'cape town',
 'santander',
 'nurota',
 'nowogard',
 'arraial do cabo',
 'gayny',
 'thinadhoo',
 'usinsk',
 'taolanaro',
 'caravelas',
 'bambous virieux',
 'prieska',
 'tel aviv-yafo',
 'mar del plata',
 'monterrey',
 'longyearbyen',
 'geraldton',
 'lebu',
 'port elizabeth',
 'abapo',
 'vertientes',
 'buraydah',
 'marcona',
 'oistins',
 'asmara',
 'villarrica',
 'novaya ladoga',
 'iskateley',
 'yining',
 'calvinia',
 'zhovtneve',
 'inhambane',
 'brae',
 'victoria',
 'tromso',
 'touros',
 'porangatu',
 'akureyri',
 'meulaboh',
 'jalu',
 'vardo',
 'roald',
 'kilindoni',
 'attawapiskat',
 'yaval',
 'bud',
 'aasiaat',
 'sibut',
 'saldanha',
 'hobyo',
 'yurginskoye',
 'chapais',
 'hermanus',
 'najran',
 'zaysan',
 'otavalo',


In [22]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [30]:
# Create an empty list to hold the weather data
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval   ")
print("---------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(1)

    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ", "+")

    # Log the URL, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count
    record_count += 1

    # Run the APO request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})
        
        # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval   
---------------------------
Processing Record 1 of Set 1 | ushuaia
Processing Record 2 of Set 1 | maragogi
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | namibe
Processing Record 5 of Set 1 | long branch
Processing Record 6 of Set 1 | santa fe
Processing Record 7 of Set 1 | mitu
Processing Record 8 of Set 1 | abu dhabi
Processing Record 9 of Set 1 | busselton
Processing Record 10 of Set 1 | upernavik
Processing Record 11 of Set 1 | road town
Processing Record 12 of Set 1 | port alfred
Processing Record 13 of Set 1 | grindavik
Processing Record 14 of Set 1 | altay
Processing Record 15 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 16 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 17 of Set 1 | takoradi
Processing Record 18 of Set 1 | jamestown
Processing Record 19 of Set 1 | saint george
Processing Record 20 of Set 1 | cape town
Processing Record 21 of Set 1 | santander
Processing 

Processing Record 40 of Set 4 | dvinskoy
Processing Record 41 of Set 4 | felidhoo
City not found. Skipping...
Processing Record 42 of Set 4 | amderma
City not found. Skipping...
Processing Record 43 of Set 4 | weligama
Processing Record 44 of Set 4 | kavaratti
Processing Record 45 of Set 4 | tasbuget
City not found. Skipping...
Processing Record 46 of Set 4 | arani
Processing Record 47 of Set 4 | yenagoa
Processing Record 48 of Set 4 | de panne
Processing Record 49 of Set 4 | havre-saint-pierre
Processing Record 50 of Set 4 | diego de almagro
Processing Record 1 of Set 5 | san vicente
Processing Record 2 of Set 5 | bembereke
Processing Record 3 of Set 5 | bad sankt leonhard
City not found. Skipping...
Processing Record 4 of Set 5 | nouadhibou
Processing Record 5 of Set 5 | bathsheba
Processing Record 6 of Set 5 | gazli
Processing Record 7 of Set 5 | upington
Processing Record 8 of Set 5 | sao sebastiao
Processing Record 9 of Set 5 | birin
Processing Record 10 of Set 5 | east london
Pro

Processing Record 27 of Set 8 | hazorasp
Processing Record 28 of Set 8 | myskhako
Processing Record 29 of Set 8 | krumovgrad
Processing Record 30 of Set 8 | damienesti
Processing Record 31 of Set 8 | vestmannaeyjar
Processing Record 32 of Set 8 | taoudenni
Processing Record 33 of Set 8 | brindisi
Processing Record 34 of Set 8 | svetlogorsk
Processing Record 35 of Set 8 | tarudant
City not found. Skipping...
Processing Record 36 of Set 8 | huarmey
Processing Record 37 of Set 8 | livingstonia
Processing Record 38 of Set 8 | pitimbu
Processing Record 39 of Set 8 | zatoka
Processing Record 40 of Set 8 | nacala
Processing Record 41 of Set 8 | el cope
Processing Record 42 of Set 8 | bodden town
Processing Record 43 of Set 8 | etah
Processing Record 44 of Set 8 | iisalmi
Processing Record 45 of Set 8 | qasigiannguit
Processing Record 46 of Set 8 | kalengwa
Processing Record 47 of Set 8 | rosetta
Processing Record 48 of Set 8 | longavi
Processing Record 49 of Set 8 | de aar
Processing Record 5

Processing Record 18 of Set 12 | alta floresta
Processing Record 19 of Set 12 | soyo
Processing Record 20 of Set 12 | tubruq
City not found. Skipping...
Processing Record 21 of Set 12 | fuerte olimpo
Processing Record 22 of Set 12 | nizip
Processing Record 23 of Set 12 | idanre
Processing Record 24 of Set 12 | onguday
Processing Record 25 of Set 12 | conceicao do araguaia
Processing Record 26 of Set 12 | koslan
Processing Record 27 of Set 12 | san jose de guanipa
Processing Record 28 of Set 12 | marzuq
Processing Record 29 of Set 12 | sawakin
Processing Record 30 of Set 12 | acari
Processing Record 31 of Set 12 | rehau
Processing Record 32 of Set 12 | bagado
Processing Record 33 of Set 12 | porto murtinho
Processing Record 34 of Set 12 | rincon
Processing Record 35 of Set 12 | galle
Processing Record 36 of Set 12 | kartaly
Processing Record 37 of Set 12 | synya
Processing Record 38 of Set 12 | arlit
Processing Record 39 of Set 12 | palasa
Processing Record 40 of Set 12 | olafsvik
Proce

Processing Record 4 of Set 16 | tal
Processing Record 5 of Set 16 | sechura
Processing Record 6 of Set 16 | kilis
Processing Record 7 of Set 16 | nioro
Processing Record 8 of Set 16 | bronnoysund
Processing Record 9 of Set 16 | mayumba
Processing Record 10 of Set 16 | jurilovca
Processing Record 11 of Set 16 | buncrana
Processing Record 12 of Set 16 | athni
Processing Record 13 of Set 16 | birkeland
Processing Record 14 of Set 16 | nantucket
Processing Record 15 of Set 16 | freeport
Processing Record 16 of Set 16 | tessalit
Processing Record 17 of Set 16 | bhag
Processing Record 18 of Set 16 | rodrigues alves
Processing Record 19 of Set 16 | envira
Processing Record 20 of Set 16 | sabha
Processing Record 21 of Set 16 | makrana
Processing Record 22 of Set 16 | loiza
Processing Record 23 of Set 16 | goure
Processing Record 24 of Set 16 | chongwe
Processing Record 25 of Set 16 | cumaribo
City not found. Skipping...
Processing Record 26 of Set 16 | winchester
Processing Record 27 of Set 16

In [37]:
len(city_data)

734

In [38]:
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Ushuaia,-54.8000,-68.3000,31.66,74,40,19.57,AR,shower snow
1,Maragogi,-9.0122,-35.2225,76.93,81,70,12.03,BR,broken clouds
2,Punta Arenas,-53.1500,-70.9167,37.42,75,20,25.32,CL,few clouds
3,Namibe,-15.1961,12.1522,63.73,87,11,4.14,AO,few clouds
4,Long Branch,40.3043,-73.9924,99.23,59,75,9.22,US,broken clouds
...,...,...,...,...,...,...,...,...,...
729,Listvyanskiy,54.4399,83.4984,59.18,96,51,7.45,RU,broken clouds
730,Charlestown,-32.9500,151.6667,61.74,82,99,5.99,AU,overcast clouds
731,Uthal,25.8072,66.6219,84.04,83,74,10.25,PK,broken clouds
732,Gariaband,20.6350,82.0614,75.45,95,100,6.15,IN,overcast clouds


In [39]:
# Rearrange order of data frame columns
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", 
                    "Humidity", "Cloudiness", "Wind Speed", "Current Description"]

city_data_df[new_column_order]

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ushuaia,AR,-54.8000,-68.3000,31.66,74,40,19.57,shower snow
1,Maragogi,BR,-9.0122,-35.2225,76.93,81,70,12.03,broken clouds
2,Punta Arenas,CL,-53.1500,-70.9167,37.42,75,20,25.32,few clouds
3,Namibe,AO,-15.1961,12.1522,63.73,87,11,4.14,few clouds
4,Long Branch,US,40.3043,-73.9924,99.23,59,75,9.22,broken clouds
...,...,...,...,...,...,...,...,...,...
729,Listvyanskiy,RU,54.4399,83.4984,59.18,96,51,7.45,broken clouds
730,Charlestown,AU,-32.9500,151.6667,61.74,82,99,5.99,overcast clouds
731,Uthal,PK,25.8072,66.6219,84.04,83,74,10.25,broken clouds
732,Gariaband,IN,20.6350,82.0614,75.45,95,100,6.15,overcast clouds


In [1]:
# Create the output file (CSV)
output_data_file = "WeatherPy_Database.csv"
# Export the city data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")

NameError: name 'city_data_df' is not defined